# ATE and ATT Estimation with Double Machine Learning

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import scipy as sp
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.metrics import mean_squared_error, log_loss
import sklearn
import os

In [4]:
RANDOM_SEED=42
np.random.seed(RANDOM_SEED)

##Load and Format Our Data (including treatment and instrument variables)

In [5]:
def make_data_covid_lockdown(df):
    df_new = df.copy()
    df_new['treatment'] = (df['tenure'] >= 3).astype(int)
    df_new['outcome'] = (df['xc_lockdown']).astype(int)
    return df_new

col_names = ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'ct_shortname', 'prov',
              'citycode', 'ctnm', 'centlon', 'centlat', 'locked_down',
              'lockdown_date', 'bdidx_19m20', 'xc_lockdown', 'xc_closed',
              'daySinceFirstCase', 'sub_prov_ct', 'gdp2018', '自治州-盟-地区', 'in_291',
              'pdensity', 'gdp_p', 'hospital_d', 'popHR18_all', 'Log_popHR18_all',
              'gdp_per_10k', 'primary_ind', 'second_ind', 'third_ind',
              'prov_leader_rank', 'num_hospital_total', 'num_doctors_total',
              'num_firm_total', 'non_domestic_firms_total',
              'pct_of_non_domestic_firm', 'primary_emp_share_total',
              'secondary_emp_share_total', 'tertiary_emp_share_total', 'inaug_time',
              'birthmonth', 'is_female', 'age_feb20', 'party_age', 'work_age',
              'tenure', 'majorchara', 'is_STEM_major', 'rule_in_native_prov', 'is_BA',
              'is_MA', 'is_PhD', 'cumulative_case', 'log_cumulative_case',
              'bdidx_19m20_feb1_10', 'lockdown_datenum', 'xc_lockdown_datenum',
              'xc_closed_datenum', 'hu_liao_jiang_nei', 'mayor', 'mayor_age',
              'mayor_work_age', 'xunshi', 'treatment', 'outcome']

dataset = pd.read_csv("/content/drive/MyDrive/stat274/IVdata-1.csv")
data = make_data_covid_lockdown(dataset)

In [6]:
data.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,ct_shortname,prov,citycode,ctnm,centlon,centlat,locked_down,lockdown_date,bdidx_19m20,xc_lockdown,xc_closed,daySinceFirstCase,sub_prov_ct,gdp2018,自治州-盟-地区,in_291,pdensity,gdp_p,hospital_d,popHR18_all,Log_popHR18_all,gdp_per_10k,primary_ind,second_ind,third_ind,prov_leader_rank,num_hospital_total,num_doctors_total,num_firm_total,non_domestic_firms_total,pct_of_non_domestic_firm,primary_emp_share_total,secondary_emp_share_total,tertiary_emp_share_total,inaug_time,birthmonth,is_female,age_feb20,party_age,work_age,tenure,majorchara,is_STEM_major,rule_in_native_prov,is_BA,is_MA,is_PhD,cumulative_case,log_cumulative_case,bdidx_19m20_feb1_10,lockdown_datenum,xc_lockdown_datenum,xc_closed_datenum,hu_liao_jiang_nei,mayor,mayor_age,mayor_work_age,xunshi,treatment,outcome
0,0,0,0,七台河,黑龙江省,230900,七台河市,130.862488,45.896891,0,NaN,3.8118,0.0,0.0,NaN,0,1757953.0,False,True,133.42,2.6500,0.1789,79.0,4.369448,22252.569620,8.79,41.54,49.66,31.47,27.0,1789.0,83.0,1.0,1.204819,4.66,50.76,44.58,2018-12,1965-1,0,55,34.0,32.0,1,2.0,1.0,1,0,0,1,17.0,2.833213,3.24543,30.0,30.0,30.0,0,贾君,56,38,1,0,0
1,1,1,1,三亚,海南省,460200,三亚市,109.418152,18.391594,0,NaN,2.9093,0.0,1.0,NaN,0,5298048.0,False,True,301.93,6.3273,0.2219,59.0,4.077537,89797.423729,12.39,19.98,67.63,32.74,11.0,2219.0,21.0,5.0,23.809524,2.79,8.12,89.09,2018-11,1967-10,0,52,15.0,28.0,1,1.0,0.0,0,0,1,0,54.0,3.988984,1.95423,30.0,30.0,5.0,0,阿东,49,22,1,0,0
2,2,2,2,三明,福建省,350400,三明市,117.395196,26.299947,0,NaN,2.5189,0.0,0.0,NaN,0,4532584.0,False,True,123.67,7.3261,0.5439,288.0,5.662960,15738.138889,2.97,54.36,42.67,35.30,46.0,5439.0,1770.0,75.0,4.237288,1.54,35.39,63.07,2019-3,1962-12,0,57,35.0,38.0,0,1.0,0.0,1,0,1,0,14.0,2.639057,2.03364,30.0,30.0,30.0,0,余红胜,49,29,1,0,0
3,3,3,3,三门峡,河南省,411200,三门峡市,111.106682,34.362922,0,NaN,2.8705,0.0,1.0,NaN,0,4316182.0,False,True,217.23,5.8894,0.5297,228.0,5.429346,18930.622807,5.85,45.02,49.12,55.64,51.0,5297.0,501.0,14.0,2.794411,0.30,50.95,48.75,2016-8,1963-10,0,56,34.0,32.0,3,1.0,0.0,1,0,0,1,7.0,1.945910,2.55164,30.0,30.0,0.0,0,安伟,53,28,1,1,0
4,4,4,4,上饶,江西省,361100,上饶市,117.469239,28.774557,1,2020-02-06,4.0219,1.0,1.0,-12.0,0,6203743.0,False,True,339.61,2.6996,1.1101,783.0,6.663133,7923.043423,4.80,41.95,53.24,32.02,147.0,11101.0,1303.0,34.0,2.609363,1.27,42.54,56.19,2016-4,1962-11,0,57,36.0,37.0,3,1.0,0.0,1,0,1,0,123.0,4.812184,3.29901,6.0,6.0,6.0,1,陈云,43,20,0,1,1


In [7]:
# confounder_fields = ['age_feb20']
confounder_fields = ['sub_prov_ct', 'gdp_per_10k', 'primary_emp_share_total']

confounders = data[confounder_fields]
outcome = data['outcome']
treatment = data['treatment']

## Specify Nuisance Function Models

The next step is to specify models for the conditional expected outcome and propensity score.

In [31]:
# specify a model for the conditional expected outcome

# make a function that returns a sklearn model for later use in k-folding
def make_Q_model():
  return RandomForestRegressor(random_state=RANDOM_SEED, n_estimators=2000, max_depth=5)
  # return LinearRegression()
Q_model = make_Q_model()

# Sanity check that chosen model actually improves test error
# A real analysis should give substantial attention to model selection and validation 

X_w_treatment = confounders.copy()
X_w_treatment["treatment"] = treatment

X_train, X_test, y_train, y_test = train_test_split(X_w_treatment, outcome, test_size=0.2)
Q_model.fit(X_train, y_train)
y_pred = Q_model.predict(X_test)

test_mse=mean_squared_error(y_pred, y_test)
print(f"Test MSE of fit model {test_mse}") 
baseline_mse=mean_squared_error(y_train.mean()*np.ones_like(y_test), y_test)
print(f"Test MSE of no-covariate model {baseline_mse}")

Test MSE of fit model 0.2283243716356103
Test MSE of no-covariate model 0.23521526623221695


In [35]:
# specify a model for the propensity score

def make_g_model():
  # return LogisticRegression(max_iter=1000)
  return RandomForestClassifier(n_estimators=2000, max_depth=5)
  # return LogisticRegression()
  # return DecisionTreeClassifier()

g_model = make_g_model()
# Sanity check that chosen model actually improves test error
# A real analysis should give substantial attention to model selection and validation 

X_train, X_test, a_train, a_test = train_test_split(confounders, treatment, test_size=0.2)
g_model.fit(X_train, a_train)
a_pred = g_model.predict_proba(X_test)[:,1]

test_ce=log_loss(a_test, a_pred)
print(f"Test CE of fit model {test_ce}") 
baseline_ce=log_loss(a_test, a_train.mean()*np.ones_like(a_test))
print(f"Test CE of no-covariate model {baseline_ce}")

Test CE of fit model 0.6310654122374246
Test CE of no-covariate model 0.6572847874232058


## Use cross fitting to get predicted outcomes and propensity scores for each unit

In [36]:
# helper functions to implement the cross fitting

def treatment_k_fold_fit_and_predict(make_model, X:pd.DataFrame, A:np.array, n_splits:int):
    """
    Implements K fold cross-fitting for the model predicting the treatment A. 
    That is, 
    1. Split data into K folds
    2. For each fold j, the model is fit on the other K-1 folds
    3. The fitted model is used to make predictions for each data point in fold j
    Returns an array containing the predictions  

    Args:
    model: function that returns sklearn model (which implements fit and predict_prob)
    X: dataframe of variables to adjust for
    A: array of treatments
    n_splits: number of splits to use
    """
    predictions = np.full_like(A, np.nan, dtype=float)
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    
    for train_index, test_index in kf.split(X, A):
      X_train = X.loc[train_index]
      A_train = A.loc[train_index]
      g = make_model()
      g.fit(X_train, A_train)

      # get predictions for split
      predictions[test_index] = g.predict_proba(X.loc[test_index])[:, 1]

    assert np.isnan(predictions).sum() == 0
    return predictions


def outcome_k_fold_fit_and_predict(make_model, X:pd.DataFrame, y:np.array, A:np.array, n_splits:int, output_type:str):
    """
    Implements K fold cross-fitting for the model predicting the outcome Y. 
    That is, 
    1. Split data into K folds
    2. For each fold j, the model is fit on the other K-1 folds
    3. The fitted model is used to make predictions for each data point in fold j
    Returns two arrays containing the predictions for all units untreated, all units treated  

    Args:
    model: function that returns sklearn model (that implements fit and either predict_prob or predict)
    X: dataframe of variables to adjust for
    y: array of outcomes
    A: array of treatments
    n_splits: number of splits to use
    output_type: type of outcome, "binary" or "continuous"

    """
    predictions0 = np.full_like(A, np.nan, dtype=float)
    predictions1 = np.full_like(y, np.nan, dtype=float)
    if output_type == 'binary':
      kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    elif output_type == 'continuous':
      kf = KFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)

    # include the treatment as input feature
    X_w_treatment = X.copy()
    X_w_treatment["A"] = A

    # for predicting effect under treatment / control status for each data point 
    X0 = X_w_treatment.copy()
    X0["A"] = 0
    X1 = X_w_treatment.copy()
    X1["A"] = 1

    
    for train_index, test_index in kf.split(X_w_treatment, y):
      X_train = X_w_treatment.loc[train_index]
      y_train = y.loc[train_index]
      q = make_model()
      q.fit(X_train, y_train)

      if output_type =='binary':
        predictions0[test_index] = q.predict_proba(X0.loc[test_index])[:, 1]
        predictions1[test_index] = q.predict_proba(X1.loc[test_index])[:, 1]
      elif output_type == 'continuous':
        predictions0[test_index] = q.predict(X0.loc[test_index])
        predictions1[test_index] = q.predict(X1.loc[test_index])

    assert np.isnan(predictions0).sum() == 0
    assert np.isnan(predictions1).sum() == 0
    return predictions0, predictions1

In [37]:
g = treatment_k_fold_fit_and_predict(make_g_model, X=confounders, A=treatment, n_splits=10)

In [38]:
Q0,Q1=outcome_k_fold_fit_and_predict(make_Q_model, X=confounders, y=outcome, A=treatment, n_splits=10, output_type="continuous")

In [39]:
data_and_nuisance_estimates = pd.DataFrame({'g': g, 'Q0': Q0, 'Q1': Q1, 'A': treatment, 'Y': outcome})
data_and_nuisance_estimates.head()

,g,Q0,Q1,A,Y
0,0.322751,0.478824,0.479311,0,0
1,0.235406,0.337210,0.337131,0,0
2,0.268373,0.416053,0.433270,0,0
3,0.361641,0.524591,0.506393,1,0
4,0.443756,0.350658,0.351817,1,1


## Combine predicted values and data into estimate of ATT


In [40]:
def att_aiptw(Q0, Q1, g, A, Y, prob_t=None):
  """
  # Double ML estimator for the ATT
  This uses the ATT specific scores, see equation 3.9 of https://www.econstor.eu/bitstream/10419/149795/1/869216953.pdf
  """

  if prob_t is None:
    prob_t = A.mean() # estimate marginal probability of treatment

  tau_hat = (A*(Y-Q0) - (1-A)*(g/(1-g))*(Y-Q0)).mean()/ prob_t
  
  scores = (A*(Y-Q0) - (1-A)*(g/(1-g))*(Y-Q0) - tau_hat*A) / prob_t
  n = Y.shape[0] # number of observations
  std_hat = np.std(scores) / np.sqrt(n)

  return tau_hat, std_hat


In [41]:
def ate_aiptw(Q0, Q1, g, A, Y, prob_t=None):
  """
  # Double ML estimator for the ATE
  """

  tau_hat = (Q1 - Q0 + A*(Y-Q1)/g - (1-A)*(Y-Q0)/(1-g)).mean()
  
  scores = Q1 - Q0 + A*(Y-Q1)/g - (1-A)*(Y-Q0)/(1-g) - tau_hat
  n = Y.shape[0] # number of observations
  std_hat = np.std(scores) / np.sqrt(n)

  return tau_hat, std_hat


In [42]:
tau_hat, std_hat = att_aiptw(**data_and_nuisance_estimates)
print(f"The estimate is {tau_hat} pm {1.96*std_hat}")

The estimate is 0.012344029299472743 pm 0.1554339039846506


In [43]:
in_treated = data_and_nuisance_estimates['A']==1
treated_estimates = data_and_nuisance_estimates[in_treated]
tau_hat, std_hat = ate_aiptw(**treated_estimates)
print(f"The estimate is {tau_hat} pm {1.96*std_hat}")

The estimate is -0.010420692529775778 pm 0.43054856690373916


In [44]:
# Computing the estimate restricted to a population with only reasonable propensity scores
g = data_and_nuisance_estimates['g']
in_overlap_popluation = ((g > 0.05) & (g < 0.95))
overlap_data_and_nuisance = data_and_nuisance_estimates[in_overlap_popluation]
tau_hat, std_hat = att_aiptw(**overlap_data_and_nuisance)
print(f"The estimate is {tau_hat} pm {1.96*std_hat}")

The estimate is 0.012819440056859172 pm 0.15542787777125575
